In [ ]:
# Import required libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

# Import libraries for text pre processing.
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
# Read dataset.
df_twitter = pd.read_csv('train.csv')
print(df_twitter.head())
print(df_twitter.shape)

In [ ]:
# Plot the distribution of the target variable.
plt.pie(df_twitter['label'].value_counts(), labels=['Non-hate', 'Hate'], autopct='%1.1f%%', shadow=True, startangle=90)
plt.title('Distribution of tweets')
plt.show()

In [ ]:
# Print number of values for each class.
class_count = df_twitter['label'].value_counts()

# Print number of hate and non-hate tweets.
print('Number of hate tweets: {}'.format(class_count[1]))
print('Number of non-hate tweets: {}'.format(class_count[0]))


In [ ]:
# Check for null values.
print(df_twitter.isnull().sum())

# Drop columns that are not required.
df_twitter.drop('id',axis=1,inplace=True)

In [ ]:
# Read another dataset
df_offensive=pd.read_csv("labeled_data.csv")

# Drop columns that are not required.
df_offensive.drop(['Unnamed: 0','count','hate_speech','offensive_language','neither'],axis=1,inplace=True)
df_offensive.head()

In [ ]:
# Print number of values for each class.
class_counts = df_offensive['class'].value_counts()

# Print the names and corresponding counts for each class/target variable.
for class_label, count in class_counts.items():
    if class_label == 1:
        class_name = 'neither'
    elif class_label == 0:
        class_name = 'hate'
    elif class_label == 2:
        class_name = 'offensive'
    else:
        class_name = 'unknown'  # Handle other cases if necessary

    print(f'{class_name}: {count}')

In [ ]:
df_offensive['class'].unique()

In [ ]:
df_offensive[df_offensive['class']==0]['class']=1
df_offensive.head(5)

In [ ]:
df_offensive['class'].unique()

In [ ]:
df_offensive[df_offensive['class']==0]

In [ ]:
df_offensive["class"].replace({0: 1}, inplace=True)

In [ ]:
df_offensive['class'].unique()

In [ ]:
df_offensive[df_offensive['class']==0]

In [ ]:
df_offensive["class"].replace({2: 0}, inplace=True)
df_offensive.rename(columns ={'class':'label'}, inplace = True)
df_offensive.head()

In [ ]:
df_offensive.iloc[0]['tweet']
df_offensive.iloc[5]['tweet']

In [ ]:
frame=[df_twitter,df_offensive]
df = pd.concat(frame)
df.head()

In [ ]:
# Plot the distribution of the target variable in new dataframe.
plt.pie(df['label'].value_counts(), labels=['Non-hate', 'Hate'], autopct='%1.1f%%', shadow=True, startangle=90)
plt.title('Distribution of tweets')
plt.show()

In [ ]:
# print number of hate and non-hate tweets.
class_count = df['label'].value_counts()
print('Number of hate tweets: {}'.format(class_count[1]))
print('Number of non-hate tweets: {}'.format(class_count[0]))


In [ ]:
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))

def clean_text(text):
    """
    Cleans the given text by converting it to lowercase, removing symbols in square brackets,
    removing URLs, removing HTML tags, removing punctuation, removing newline characters,
    removing words containing numbers, removing stopwords, and stemming words.

    Args:
        text (str): The text to be cleaned.

    Returns:
        str: The cleaned text.
    """
    text = str(text).lower() # Convert to lowercase.
    text = re.sub('\[.*?\]', '', text) # Remove symbols in square brackets.
    text = re.sub('https?://\S+|www\.\S+', '', text) # Remove URLs.
    text = re.sub('<.*?>+', '', text) # Remove HTML tags.
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation.
    text = re.sub('\n', '', text) # Remove newline characters.
    text = re.sub('\w*\d\w*', '', text) # Remove words containing numbers.
    text = [word for word in text.split(' ') if word not in stopword] # Remove stopwords.
    text=" ".join(text) # Join list of words in to a string separated by space.
    text = [stemmer.stem(word) for word in text.split(' ')] # Stem words.
    text=" ".join(text) # Join list of words in to a string separated by space.
    return text

# Clean the text in the dataset.
df['tweet']=df['tweet'].apply(clean_text)

In [ ]:
# Print the last 5 rows of the dataset.
df.tail()

In [ ]:
# Word cloud for non-hate tweets.
from wordcloud import WordCloud
plt.figure(figsize=(20,20))
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate(" ".join(df[df.label == 0].tweet))
plt.imshow(wc , interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Input data.
x=df['tweet']

# Target variable.
y=df['label']

# Split the data into training and testing sets.
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

In [ ]:
# Vectorize the text data using CountVectorizer.
count = CountVectorizer(stop_words='english', ngram_range=(1,5))

# Fit and transform the training data using CountVectorizer.
x_train_vectorizer=count.fit_transform(x_train)
# Transform the test data using the fitted CountVectorizer.
x_test_vectorizer=count.transform(x_test)
# Convert the training data to a dense array representation.
x_train_vectorizer.toarray()

In [ ]:
# Transform data using TF-IDF transformer.
tfidf = TfidfTransformer()

# Fit and transform the training data using TF-IDF transformer.
x_train_tfidf = tfidf.fit_transform(x_train_vectorizer)
# Transform the test data using the fitted TF-IDF transformer.
x_train_tfidf.toarray()
x_test_tfidf = tfidf.transform(x_test_vectorizer)

Train and build model

In [ ]:
# Tokenize the text data.
max_words = 50000
max_len = 300
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
# Convert the text data to sequence.
sequences = tokenizer.texts_to_sequences(x_train)
# Pads the sequences to ensure same length.
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
# Build a LSTM model.
model = Sequential()
model.add(Embedding(max_words, 100, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# Compile the model.
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
# Define early stopping to prevent overfitting.
es = EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=5
)

In [ ]:
# Train the model.
history=model.fit(sequences_matrix,y_train,batch_size=1024,epochs=10,
          validation_split=0.2,callbacks=[es])

In [ ]:
# Process the test data by converting it into sequences and padding them.
test_sequences = tokenizer.texts_to_sequences(x_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
# Check accuracy of model on test data.
accuracy = model.evaluate(test_sequences_matrix,y_test)

In [ ]:
lstm_prediction=model.predict(test_sequences_matrix)

In [ ]:
# Making a single prediction
new_tweet = 'I hate you'
new_tweet = clean_text(new_tweet)
new_tweet = tokenizer.texts_to_sequences([new_tweet])
new_tweet = sequence.pad_sequences(new_tweet, maxlen=max_len)
prediction = model.predict(new_tweet)
if(prediction>0.5):
    print("Hate speech")
else:
    print("Non hate speech")
    


In [ ]:
# Save the model
model.save('lstm_model.h5')

In [ ]:
# Plot the accuracy and loss of the model.
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot the training and validation loss.
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'], loc='upper right')
plt.show()

In [ ]:
# Print the confusion matrix using seaborn and colors.
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(y_test, lstm_prediction.round())
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix')
plt.show()



In [ ]:
# Print f1 score.
from sklearn.metrics import f1_score
print('F1 score: {}'.format(f1_score(y_test, lstm_prediction.round())))

# Print precision score.
from sklearn.metrics import precision_score
print('Precision score: {}'.format(precision_score(y_test, lstm_prediction.round())))

# Print recall score.
from sklearn.metrics import recall_score
print('Recall score: {}'.format(recall_score(y_test, lstm_prediction.round())))



In [ ]:
# Print ROC curve.
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, lstm_prediction)
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr,tpr, label='LSTM')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()


In [ ]:
# Print the auc score.
from sklearn.metrics import auc
auc_score=auc(fpr,tpr)
print('AUC score: {}'.format(auc_score))



In [ ]:
# Print evaluation metrics.
from sklearn.metrics import classification_report
report = classification_report(y_test, lstm_prediction.round(), target_names = ['Non-hate','Hate'])
print(report)